# Part I. ETL Pipeline for Pre-Processing the Files

* The PART I reads a list of CSV files from the workspace.
* Select particular data from those files to create **event_datafile_new.csv**
* The output of this ETL process (event_datafile_new.csv) will be used for further Data Modelling.


#### Import Python packages 

In [92]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [93]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [94]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
        

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


# Part II.  Apache Cassandra Data Modelling

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Preliminary Steps : Create Apache Cassandra Cluster and Keyspace

#### Create a Cluster and connect the session with Localhost

In [95]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()

except Exception as e:
    print(e)

#### Create Keyspace Project1

In [96]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project1 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
    
except Exception as e:
    print(e)

#### Set Keyspace

In [97]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('project1')
except Exception as e:
    print(e)

## Modelling the Database to answer 3 queries

### Query 1. Get the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


* To get the results the Partition Key should be sessionId and clustering key is itemInSession
* The Primary key is therefore (sessionId, itemInSession) 
* This is not a composite primary key

Note: This could be solved with a composite primary key ((sessionId, itemInSession)). 

But in that case, we could **not use WHERE clause on sessionId ONLY**
(This is important to Verify that the previous output has Faithless Music Matters at itemInSession = 4 )

#### Build the Data Model based on Query 1

In [107]:

query = "CREATE TABLE IF NOT EXISTS artist_song"
query = query + "(sessionId int, itemInSession int, artist text, firstName text, gender text, lastName text, length float, \
 level text, location text, song text, userId int, PRIMARY KEY (sessionId, itemInSession) )"
try:
    session.execute(query)
except Exception as e:
    print(e)            

In [108]:
file = 'event_datafile_new.csv'

#open the CSV file in read mode and insert the data from CSV file to Cassandra table artist_song

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song (sessionId, itemInSession, artist, firstName, gender, lastName, length, level,\
                 location, song, userId)"
    
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                

        session.execute(query, (int(line[8]), int(line[3]), line[0], line[1], line[2], line[4], float(line[5]),  \
                                 line[6], line[7], line[9], int(line[10])))
        
        
# Run the query to get the results ( refer to the Query 1)      
query = "select artist, song, length from artist_song  WHERE sessionId = 338 AND itemInSession = 4;"


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Verify that the previous output has Faithless Music Matters at itemInSession = 4

The order of sort is based on Clustering Key itemInSession

In [109]:
query = "select iteminsession, artist, song, length from artist_song  where sessionId = 338;"


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.iteminsession, row.artist, row.song, row.length)


1 Pixies Build High 89.36444091796875
2 The Roots / Jack Davey Atonement 155.95057678222656
3 Mike And The Mechanics A Beggar On A Beach Of Gold 275.1211853027344
4 Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2. Get name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

* To get the results the Partition Key should be userId and clustering keys are sessionId, itemInSession
* The Primary key is therefore (userId, sessionId, itemInSession ) 
* The clustering column itemInSession sort the output

In [110]:
query = "CREATE TABLE IF NOT EXISTS artist_username_session"
query = query + "(userId int, sessionId int, itemInSession int, artist text, firstName text, gender text, lastName text, \
 length float, level text, location text,  song text,  PRIMARY KEY (userId, sessionId, itemInSession ))"
try:
    session.execute(query)
except Exception as e:
    print(e)    
                    

In [111]:
file = 'event_datafile_new.csv'

#open the CSV file in read mode and insert the data from CSV file to Cassandra table artist_username_session

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_username_session (userId,  sessionId,  itemInSession, artist, firstName, gender, lastName,\
                  length, level,location, song)"
    
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[2], line[4], float(line[5]),  \
                                line[6], line[7], line[9]))
        

# Run the query to get the results ( refer to the Query 2)
        
query = "select artist, song, firstname, lastname from artist_username_session  WHERE userid = 10 AND sessionid = 182;"



try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Verify the order has been maintained (sorted by iteminsession)

In [112]:
query = "select iteminsession, artist, song, firstname, lastname from artist_username_session  where userid = 10 AND sessionid = 182;"


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz



### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

* To get the results the Partition Key should be song and clustering keys are firstName, sessionId, itemInSession
* The Primary key is therefore (song, firstName, sessionId, itemInSession) 
* Included the sessionId, itemInSession to make each record unique.

In [113]:
query = "CREATE TABLE IF NOT EXISTS song_user_session"
query = query + "(song text, firstName text, sessionId int, itemInSession int, artist text,  gender text,  lastName text, \
 length float, level text, location text, userId int, PRIMARY KEY (song, firstName, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)    
                   
                    

In [114]:
file = 'event_datafile_new.csv'


#open the CSV file in read mode and insert the data from CSV file to Cassandra table song_user_session

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_session (song,  firstName, sessionId, itemInSession, artist, gender, lastName, length, level,\
                 location, userId)"
    
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (line[9], line[1], int(line[8]), int(line[3]), line[0], line[2], line[4], \
                                  float(line[5]), line[6], line[7],int(line[10])))
    

# Run the query to get the results ( refer to the Query 3)

query = "select firstname, lastname from song_user_session  WHERE song = 'All Hands Against His Own' ;"


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ( row.firstname, row.lastname)

Jacqueline Lynch
Sara Johnson
Tegan Levine


#### Verify that the song is 'All Hands Against HIs Own' for all previous output

In [115]:
query = "select firstname, lastname, song from song_user_session  WHERE song = 'All Hands Against His Own';"


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname, row.song)

Jacqueline Lynch All Hands Against His Own
Sara Johnson All Hands Against His Own
Tegan Levine All Hands Against His Own


### Droping the tables before closing out the sessions

In [116]:
query = "drop table artist_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


query = "drop table artist_username_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    
query = "drop table song_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [117]:
session.shutdown()
cluster.shutdown()